# Proyecto 1 — Limpieza (v3, completo y reproducible)

Este cuaderno aplica **solo limpieza** (sin análisis) a los establecimientos **NIVEL DIVERSIFICADO** y deja constancia de cada paso.  
Incluye:
- Normalización de textos (mayúsculas y sin acentos), espacios y guiones, **sin eliminar guiones internos válidos**.
- Expansión de abreviaturas frecuentes en `DIRECCION` (AV., CALZ., Z., KM., No./N°, etc.).
- Estandarización de `TELEFONO` (múltiples números de 8 dígitos separados por ` / `).
- **Corrección OCR específica para `RUTA`**: I/L→1, O→0 **antes de dígitos** (cubre todos los dígitos 0–9, con o sin espacio).
- **ZONA**: crea/asegura columna, extrae de `DEPARTAMENTO`/`MUNICIPIO`/`DIRECCION`; normaliza **CIUDAD CAPITAL** a `GUATEMALA/GUATEMALA` moviendo ZONA.
- Detección **multicriterio** de duplicados: ESTRICTO, SOFT1, SOFT2, SOFT3 + etiqueta `TIPO_DUP`.
- **Bitácora** con cada acción.

> **Entradas:** Un único CSV consolidado (por defecto `establecimientos_diversificado_raw_concat.csv`).  
> **Salidas:** CSV limpio v3, duplicados v3, resumen duplicados v3, bitácora v3, y libro de códigos v3 (Markdown).


In [ ]:
# === Configuración de rutas ===
INPUT_CSV  = "establecimientos_diversificado_raw_concat.csv"  # cambia aquí si tu archivo tiene otro nombre
CLEAN_CSV  = "establecimientos_diversificado_limpio_v3.csv"
DUPS_CSV   = "duplicados_v3.csv"
SUMMARY_CSV= "resumen_duplicados_v3.csv"
LOG_CSV    = "bitacora_limpieza_v3.csv"
CODEBOOK_MD= "Libro_de_Codigos_Proyecto1_v3.md"  # generado al final (opcional)

# Si el archivo está junto al cuaderno, no cambies nada. Si está en otra carpeta, pon la ruta completa.


In [ ]:
import re, os, unicodedata
from datetime import datetime
import pandas as pd
import numpy as np

pd.set_option("display.max_colwidth", 120)


## 1) Carga de datos y nombres de columnas en MAYÚSCULAS

In [ ]:
# Cargar SIN inferir tipos (todo como str) y normalizar nombres de columnas
df = pd.read_csv(INPUT_CSV, dtype=str)
df.columns = [c.upper() for c in df.columns]
df.shape, df.columns.tolist()[:20]

## 2) Utilidades de normalización

In [ ]:
def strip_accents(s: str) -> str:
    if not isinstance(s, str): return s
    return "".join(ch for ch in unicodedata.normalize("NFKD", s) if not unicodedata.combining(ch))

def collapse_spaces_and_hyphens(s: str) -> str:
    if not isinstance(s, str): return s
    s = s.replace("—","-").replace("–","-").replace("−","-").replace("­","")
    s = re.sub(r"[ \t\u00A0]+", " ", s)   # colapsa espacios
    s = re.sub(r"-{2,}", "-", s)          # 2+ guiones -> uno
    s = re.sub(r"\s*-\s*", " - ", s)      # espacios estándar alrededor del guion
    s = re.sub(r"\s{2,}", " ", s)         # colapsa dobles espacios
    return s.strip()

def normalize_text_basic(s: str) -> str:
    if not isinstance(s, str): return s
    s = s.strip().replace('"', "").replace("“","").replace("”","").replace("´", "'")
    s = strip_accents(s).upper()
    s = collapse_spaces_and_hyphens(s)
    return s

def remove_leading_bullets(s: str) -> str:
    if not isinstance(s, str): return s
    # elimina bullets iniciales (p.ej. "- IGA", "-IGA", "• NOMBRE")
    return re.sub(r"^\s*[-•]+\s*", "", s)

def expand_address_abbrev(s: str) -> str:
    if not isinstance(s, str): return s
    txt = " " + s + " "
    rules = [
        (r"\bAV[\.]?\b", " AVENIDA "), (r"\bAVE[\.]?\b", " AVENIDA "), (r"\bAVDA[\.]?\b", " AVENIDA "),
        (r"\bBLVD[\.]?\b", " BOULEVARD "), (r"\bCALZ[\.]?\b", " CALZADA "),
        (r"\bCOL[\.]?\b", " COLONIA "), (r"\bCOND[\.]?\b", " CONDOMINIO "), (r"\bRES[\.]?\b", " RESIDENCIAL "),
        (r"\bZ[\.]?\b(?=\s*\d)", " ZONA "), (r"\bZONA\b\s*(?=\d)", " ZONA "),
        (r"\bKM[\.]?\b", " KM "), (r"\bNO[\.]?\b", " NUMERO "), (r"\bN[\.]?\b(?=\s*\d)", " NUMERO "), (r"\b#\b", " NUMERO "),
        (r"\bEDIF[\.]?\b", " EDIFICIO "),
        (r"\b2DA\b", " 2A "), (r"\b3RA\b", " 3A ")
    ]
    for pat, rep in rules:
        txt = re.sub(pat, rep, txt, flags=re.IGNORECASE)
    txt = collapse_spaces_and_hyphens(txt)
    return txt

def normalize_name(s: str) -> str:
    if not isinstance(s, str): return s
    s = remove_leading_bullets(s)
    s = normalize_text_basic(s)
    return s

def normalize_address(s: str) -> str:
    if not isinstance(s, str): return s
    s = remove_leading_bullets(s)
    s = normalize_text_basic(s)
    s = expand_address_abbrev(s)
    # casos como "2. CALLE" -> "2 CALLE"
    s = re.sub(r"\b(\d+)\s*\.\s*", r"\1 ", s)
    s = re.sub(r"\bKM\.\b", " KM ", s)
    s = collapse_spaces_and_hyphens(s)
    return s

def clean_phone_field(s: str):
    """Extrae grupos de 8 dígitos; ignora extensiones; retorna ' / ' separado o NA."""
    if not isinstance(s, str): return None
    nums = re.findall(r"\d{8}", s)
    nums = [n for n in nums if not re.fullmatch(r"0{8}", n)]
    seen, unique = set(), []
    for n in nums:
        if n not in seen:
            seen.add(n); unique.append(n)
    return " / ".join(unique) if unique else None

def make_dup_key(nombre, direccion, municipio, departamento) -> str:
    nombre_n = normalize_name(nombre) if isinstance(nombre, str) else ""
    direccion_n = normalize_address(direccion) if isinstance(direccion, str) else ""
    municipio_n = normalize_text_basic(municipio) if isinstance(municipio, str) else ""
    depto_n = normalize_text_basic(departamento) if isinstance(departamento, str) else ""
    key = f"{nombre_n} | {direccion_n} | {municipio_n} | {depto_n}"
    return collapse_spaces_and_hyphens(key)


## 3) Limpieza básica + backups de originales

In [ ]:
bitacora = []
def log(accion: str, justificacion: str, resumen=None):
    bitacora.append({
        "timestamp": datetime.now().isoformat(timespec="seconds"),
        "accion": accion,
        "justificacion": justificacion,
        "resumen": resumen or {}
    })

# Normalización básica en todas las columnas tipo texto
obj_cols = [c for c in df.columns if df[c].dtype == "object"]
df[obj_cols] = df[obj_cols].apply(lambda s: s.map(lambda x: collapse_spaces_and_hyphens(x) if isinstance(x, str) else x))

# Estandarizar vacíos/guiones
df[obj_cols] = df[obj_cols].replace({"": pd.NA, "nan": pd.NA, "None": pd.NA, "-": pd.NA, "--": pd.NA, "—": pd.NA})

# Backups originales
for col in ["ESTABLECIMIENTO","DIRECCION","TELEFONO"]:
    if col in df.columns and f"{col}_ORIG" not in df.columns:
        df[f"{col}_ORIG"] = df[col]

# Bullets iniciales y placeholders
for col in ["ESTABLECIMIENTO","DIRECCION"]:
    if col in df.columns: df[col] = df[col].map(remove_leading_bullets)

placeholders = re.compile(r"^[-–—]{1,}$")
for col in ["ESTABLECIMIENTO","DIRECCION","TELEFONO","MUNICIPIO","DEPARTAMENTO"]:
    if col in df.columns:
        df.loc[df[col].astype(str).str.fullmatch(placeholders, na=False), col] = pd.NA

log("limpieza_basica", "Colapsar espacios/guiones; estandarizar vacíos; quitar bullets y placeholders.")
df.shape

## 4) Normalización de campos clave

In [ ]:
# Nombre y dirección (mayúsculas sin acentos, abreviaturas dirección)
if "ESTABLECIMIENTO" in df.columns:
    df["ESTABLECIMIENTO"] = df["ESTABLECIMIENTO"].map(lambda x: normalize_name(x) if isinstance(x, str) else x)
if "DIRECCION" in df.columns:
    df["DIRECCION"] = df["DIRECCION"].map(lambda x: normalize_address(x) if isinstance(x, str) else x)

# Teléfono (8 dígitos, múltiples con ' / ')
if "TELEFONO" in df.columns:
    df["TELEFONO"] = df["TELEFONO"].map(lambda x: clean_phone_field(x) if isinstance(x, str) else x)
    df["TELEFONO_VALIDO"] = df["TELEFONO"].apply(lambda x: bool(x) if pd.notna(x) else False)

# Categóricas a mayúsculas sin acentos
for col in ["DEPARTAMENTO","MUNICIPIO","SECTOR","AREA","STATUS","MODALIDAD","JORNADA","PLAN","NIVEL"]:
    if col in df.columns:
        df[col] = df[col].map(lambda x: normalize_text_basic(x) if isinstance(x, str) else x)

log("normalizar_campos", "Normalizar ESTABLECIMIENTO, DIRECCION, TELEFONO y categóricas.")
df.head(3)

## 5) Corrección OCR en `RUTA` (I/L→1 y O→0 ante dígito)

In [ ]:
def fix_ruta_ocr_general(s: str) -> str:
    if not isinstance(s, str): return s
    txt = s
    # I o L (may/min) como 1 cuando van antes de un dígito
    txt = re.sub(r"\b(RUTA\s+)[ILil]\s*(\d)\b", r"\g<1>1\2", txt)
    # O (may/min) como 0 cuando va antes de un dígito
    txt = re.sub(r"\b(RUTA\s+)[Oo]\s*(\d)\b", r"\g<1>0\2", txt)
    return txt

if "DIRECCION" in df.columns:
    df["DIRECCION"] = df["DIRECCION"].map(fix_ruta_ocr_general)

log("ruta_ocr", "Corrección OCR general en RUTA (I/L->1, O->0).")
df.loc[df["DIRECCION"].fillna("").str.contains(r"\\bRUTA\\b"), ["ESTABLECIMIENTO","DIRECCION"]].head(5)

## 6) Columna `ZONA` + normalización de **CIUDAD CAPITAL**

In [ ]:
def extract_zona(val: str):
    if not isinstance(val, str): return None
    m = re.search(r"\bZONA\s*(\d{1,2})\b", val, flags=re.IGNORECASE)
    if m:
        return (m.group(1).lstrip("0") or "0")
    return None

if "ZONA" not in df.columns:
    df["ZONA"] = pd.NA

def normalize_capital_and_zone(row):
    dept = str(row.get("DEPARTAMENTO", "") or "")
    muni = str(row.get("MUNICIPIO", "") or "")
    dire = str(row.get("DIRECCION", "") or "")

    z_depto = extract_zona(dept)
    z_muni  = extract_zona(muni)
    z_dir   = extract_zona(dire)

    is_capital = bool(re.search(r"\bCIUDAD\s+CAPITAL\b", dept, re.IGNORECASE) or
                      re.search(r"\bCIUDAD\s+CAPITAL\b", muni, re.IGNORECASE))

    if z_depto is not None or z_muni is not None or is_capital:
        row["DEPARTAMENTO"] = "GUATEMALA"
        row["MUNICIPIO"] = "GUATEMALA"
        zona = z_depto or z_muni or z_dir
        if zona is not None:
            row["ZONA"] = zona
    else:
        if z_dir is not None:
            row["ZONA"] = z_dir
    return row

df = df.apply(normalize_capital_and_zone, axis=1)
df["ZONA"] = df["ZONA"].where(df["ZONA"].notna() & df["ZONA"].astype(str).str.strip().ne(""), pd.NA)

log("zona_capital", "Asegurar ZONA; mover zonas de dept/muni a ZONA y normalizar CIUDAD CAPITAL a GUATEMALA/GUATEMALA.")
df[["DEPARTAMENTO","MUNICIPIO","ZONA"]].head(5)

## 7) Claves de duplicado: ESTRICTO y 3 suaves (+ `TIPO_DUP`)

In [ ]:
def key_strict(r):
    return collapse_spaces_and_hyphens(f"{normalize_name(r.get('ESTABLECIMIENTO',''))} | "
                                       f"{normalize_address(r.get('DIRECCION',''))} | "
                                       f"{normalize_text_basic(r.get('MUNICIPIO',''))} | "
                                       f"{normalize_text_basic(r.get('DEPARTAMENTO',''))}")

def key_soft1(r):
    return collapse_spaces_and_hyphens(f"{normalize_name(r.get('ESTABLECIMIENTO',''))} | "
                                       f"{normalize_text_basic(r.get('MUNICIPIO',''))} | "
                                       f"{normalize_text_basic(r.get('DEPARTAMENTO',''))} | "
                                       f"{normalize_text_basic(r.get('JORNADA',''))}")

def key_soft2(r):
    return collapse_spaces_and_hyphens(f"{normalize_name(r.get('ESTABLECIMIENTO',''))} | "
                                       f"{normalize_text_basic(r.get('MUNICIPIO',''))} | "
                                       f"{normalize_text_basic(r.get('DEPARTAMENTO',''))}")

def key_soft3(r):
    return collapse_spaces_and_hyphens(f"{normalize_name(r.get('ESTABLECIMIENTO',''))} | "
                                       f"{normalize_text_basic(r.get('DEPARTAMENTO',''))} | "
                                       f"{normalize_text_basic(r.get('JORNADA',''))} | "
                                       f"{normalize_text_basic(r.get('PLAN',''))}")

df["CLAVE_DUP_ESTRICTO"] = df.apply(key_strict, axis=1)
df["CLAVE_DUP_SOFT1"]    = df.apply(key_soft1,  axis=1)
df["CLAVE_DUP_SOFT2"]    = df.apply(key_soft2,  axis=1)
df["CLAVE_DUP_SOFT3"]    = df.apply(key_soft3,  axis=1)

for colkey, flagname in [
    ("CLAVE_DUP_ESTRICTO", "DUP_ESTRICTO"),
    ("CLAVE_DUP_SOFT1",    "DUP_SOFT1"),
    ("CLAVE_DUP_SOFT2",    "DUP_SOFT2"),
    ("CLAVE_DUP_SOFT3",    "DUP_SOFT3"),
]:
    sizes = df.groupby(colkey)[colkey].transform("size")
    df[flagname] = sizes.gt(1)

def tipo_dup_row(row):
    tipos = []
    if row.get("DUP_ESTRICTO", False): tipos.append("ESTRICTO")
    if row.get("DUP_SOFT1",   False): tipos.append("SOFT1")
    if row.get("DUP_SOFT2",   False): tipos.append("SOFT2")
    if row.get("DUP_SOFT3",   False): tipos.append("SOFT3")
    return ", ".join(tipos) if tipos else ""

df["TIPO_DUP"] = df.apply(tipo_dup_row, axis=1)
df["DUP_ANY"]  = df[["DUP_ESTRICTO","DUP_SOFT1","DUP_SOFT2","DUP_SOFT3"]].any(axis=1)

log("duplicados", "Marcaje de duplicados: estricto y suaves (SOFT1/2/3).")
df.loc[df["DUP_ANY"], ["ESTABLECIMIENTO","MUNICIPIO","DEPARTAMENTO","JORNADA","PLAN","TIPO_DUP"]].head(5)

## 8) Guardado de salidas

In [ ]:
# Guardar dataset limpio v3
df.to_csv(CLEAN_CSV, index=False, encoding="utf-8")

# Guardar duplicados unificado
cols_base = ["CODIGO","ESTABLECIMIENTO","DIRECCION","ZONA","TELEFONO","MUNICIPIO","DEPARTAMENTO","JORNADA","PLAN"]
cols_keys = ["CLAVE_DUP_ESTRICTO","CLAVE_DUP_SOFT1","CLAVE_DUP_SOFT2","CLAVE_DUP_SOFT3","TIPO_DUP"]
dups_df = df.loc[df["DUP_ANY"], cols_base + cols_keys].sort_values(["DEPARTAMENTO","MUNICIPIO","ESTABLECIMIENTO","DIRECCION"])
dups_df.to_csv(DUPS_CSV, index=False, encoding="utf-8")

# Resumen duplicados
resumen = pd.DataFrame({
    "tipo": ["ESTRICTO","SOFT1","SOFT2","SOFT3","ANY"],
    "filas": [
        int(df["DUP_ESTRICTO"].sum()),
        int(df["DUP_SOFT1"].sum()),
        int(df["DUP_SOFT2"].sum()),
        int(df["DUP_SOFT3"].sum()),
        int(df["DUP_ANY"].sum()),
    ],
    "clusters": [
        int(df.loc[df["DUP_ESTRICTO"], "CLAVE_DUP_ESTRICTO"].nunique()),
        int(df.loc[df["DUP_SOFT1"],    "CLAVE_DUP_SOFT1"].nunique()),
        int(df.loc[df["DUP_SOFT2"],    "CLAVE_DUP_SOFT2"].nunique()),
        int(df.loc[df["DUP_SOFT3"],    "CLAVE_DUP_SOFT3"].nunique()),
        int(len(set().union(
            df.loc[df["DUP_ESTRICTO"], "CLAVE_DUP_ESTRICTO"].unique().tolist(),
            df.loc[df["DUP_SOFT1"],    "CLAVE_DUP_SOFT1"].unique().tolist(),
            df.loc[df["DUP_SOFT2"],    "CLAVE_DUP_SOFT2"].unique().tolist(),
            df.loc[df["DUP_SOFT3"],    "CLAVE_DUP_SOFT3"].unique().tolist(),
        ))),
    ]
})
resumen.to_csv(SUMMARY_CSV, index=False, encoding="utf-8")

# Bitácora
pd.DataFrame(bitacora).to_csv(LOG_CSV, index=False, encoding="utf-8")

{
    "clean": CLEAN_CSV,
    "dups": DUPS_CSV,
    "summary": SUMMARY_CSV,
    "log": LOG_CSV,
    "rows": len(df),
    "zona_con_valor": int(df["ZONA"].notna().sum()),
    "zona_na": int(df["ZONA"].isna().sum())
}

## 9) (Opcional) Función de búsqueda rápida (robusta a acentos/guiones)

In [ ]:
import unicodedata

def _norm(s):
    if not isinstance(s, str): return ""
    s = s.strip().upper()
    s = "".join(ch for ch in unicodedata.normalize("NFKD", s) if not unicodedata.combining(ch))
    s = re.sub(r"\s*-\s*", " - ", s)
    s = re.sub(r"\s+", " ", s)
    return s

def buscar(df, nombre=None, municipio=None, departamento=None, zona=None):
    m = pd.Series(True, index=df.index)
    if nombre:
        m &= df["ESTABLECIMIENTO"].fillna("").map(_norm).str.contains(re.escape(_norm(nombre)), regex=True, na=False)
    if municipio:
        m &= df["MUNICIPIO"].fillna("").map(_norm).str.contains(re.escape(_norm(municipio)), regex=True, na=False)
    if departamento:
        m &= df["DEPARTAMENTO"].fillna("").map(_norm).str.contains(re.escape(_norm(departamento)), regex=True, na=False)
    if zona is not None:
        m &= df["ZONA"].astype(str).fillna("").eq(str(zona))
    cols = ["CODIGO","ESTABLECIMIENTO","DIRECCION","ZONA","MUNICIPIO","DEPARTAMENTO","JORNADA","PLAN",
            "DUP_ESTRICTO","DUP_SOFT1","DUP_SOFT2","DUP_SOFT3","TIPO_DUP"]
    present = [c for c in cols if c in df.columns]
    return df.loc[m, present].sort_values(["DEPARTAMENTO","MUNICIPIO","ESTABLECIMIENTO","DIRECCION"]).reset_index(drop=True)

# Ejemplo:
# buscar(df, nombre="IGA", municipio="GUATEMALA")


## 10) (Opcional) Generar **Libro de Códigos** (Markdown)

In [ ]:
lines = []
lines.append("# Libro de Códigos – Establecimientos (Diversificado, v3)\n")
lines.append("**Estándares de limpieza aplicados:**\n")
lines.append("- Texto en MAYÚSCULAS y SIN ACENTOS en campos categóricos.\n")
lines.append("- Guiones internos conservados con espacios estándar (` - `).\n")
lines.append("- Bullets iniciales removidos en `ESTABLECIMIENTO`/`DIRECCION`.\n")
lines.append("- `DIRECCION`: abreviaturas expandidas (AV., AVE., CALZ., Z., KM., NO./N°. → AVENIDA, CALZADA, ZONA, KM, NUMERO).\n")
lines.append("- `TELEFONO`: solo secuencias de 8 dígitos; múltiples separados por ` / `.\n")
lines.append("- **RUTA OCR**: I/L→1, O→0 ante dígito, sólo en contexto `RUTA`.\n")
lines.append("- `ZONA`: extraída de dept/muni/dirección; `CIUDAD CAPITAL` → `GUATEMALA/GUATEMALA`.\n")
lines.append("- Duplicados: ESTRICTO y SOFT1/2/3; etiqueta `TIPO_DUP`.\n")
lines.append("- Sin eliminación de filas/columnas; con bitácora de pasos.\n")

lines.append("\n**Descripción general:**\n")
lines.append(f"- Registros: {len(df)}\n")
lines.append(f"- ZONA con valor: {int(df['ZONA'].notna().sum())} | NA: {int(df['ZONA'].isna().sum())}\n")
lines.append("- Archivos: `duplicados_v3.csv`, `resumen_duplicados_v3.csv`, `bitacora_limpieza_v3.csv`.\n")

lines.append("\n## Variables\n")
for col in df.columns:
    ejemplo = df[col].dropna().astype(str).head(3).tolist()
    ejemplo = [e[:120] for e in ejemplo]
    desc = ""
    if col == "ESTABLECIMIENTO_ORIG": desc = "Valor original del nombre (antes de limpieza)."
    elif col == "DIRECCION_ORIG": desc = "Valor original de la dirección (antes de limpieza)."
    elif col == "TELEFONO_ORIG": desc = "Valor original del teléfono (antes de extracción)."
    elif col == "TELEFONO_VALIDO": desc = "Indicador: True si se extrajo al menos un teléfono de 8 dígitos."
    elif col.startswith("CLAVE_DUP"): desc = "Clave canónica para detección de duplicados (no elimina filas)."
    elif col.startswith("DUP_"): desc = "Bandera booleana: True si el registro pertenece a un cluster de duplicados bajo este criterio."
    elif col == "TIPO_DUP": desc = "Tipos de duplicado aplicables al registro (ESTRICTO, SOFT1, SOFT2, SOFT3)."
    elif col == "ZONA": desc = "Zona administrativa (1–25 aprox. para Ciudad de Guatemala); NA si no aplica/no disponible."
    elif col == "DIRECCION": desc = "Dirección normalizada; abreviaturas expandidas; corrección OCR en RUTA."
    elif col == "ESTABLECIMIENTO": desc = "Nombre normalizado; bullets iniciales removidos; guiones internos conservados."
    elif col == "TELEFONO": desc = "Teléfonos de 8 dígitos separados por ` / `."
    elif col == "NIVEL": desc = "Nivel educativo (esperado: DIVERSIFICADO)."
    elif col in {"DEPARTAMENTO","MUNICIPIO"}: desc = "Ubicación administrativa normalizada."
    else: desc = "Campo de la fuente; normalizado a mayúsculas sin acentos cuando aplica."
    lines.append(f"### {col}\n- **Descripción:** {desc}\n- **Ejemplo(s):** {', '.join(ejemplo) if ejemplo else '—'}\n")

with open(CODEBOOK_MD, "w", encoding="utf-8") as f:
    f.write("\n".join(lines))

CODEBOOK_MD